<a href="https://colab.research.google.com/github/fdac25/brawlhalla/blob/main/usage_over_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
import plotly.express as px
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("brawlhalla_matches.csv")
data

# Getting Complete List Of All Legends Used

In [ ]:
legends = []
for row in data.itertuples(index=True):
  p1legends = list(filter(bool, [string.strip() for string in row.player_1_legends.split(",")])) # Grab P1's legends and remove whitespace/empties
  for legend in p1legends: # Iterate through all of P1's legends
    if legend not in legends: # If legend is not already recorded in legends
        legends.append(legend) # Append legend to list

  p2legends = list(filter(bool, [string.strip() for string in row.player_2_legends.split(",")])) # Grab P2's legends and remove whitespace/empties
  for legend in p2legends: # Iterate through all of P2's legends
    if legend not in legends: # If legend is not already recorded in legends
        legends.append(legend) # Append legend to list

if "Not Reported" in legends:
  legends.remove("Not Reported") # Remove 'Not Reported' from the list of legends

legends

# Calculating Win/Usage Of Each Character

In [ ]:
for legend in legends[:5]:
  uses = []
  for row in data.itertuples(index=True):
    # Record winner of the match as an integer
    if "1" in row.winner:
      winner = 1
    elif "2" in row.winner:
      winner = 2

    p1legends = list(set(list(filter(bool, [string.strip() for string in row.player_1_legends.split(",")])))) # Grab P1's legends and remove whitespace/empties/duplicates
    if legend in p1legends:
      uses.append([row.match_id, row.date, legend, winner == 1])

    p2legends = list(set(list(filter(bool, [string.strip() for string in row.player_2_legends.split(",")])))) # Grab P2's legends and remove whitespace/empties/duplicates
    if legend in p2legends:
      uses.append([row.match_id, row.date, legend, winner == 2])
  usesdf = pd.DataFrame(uses, columns=['match_id', 'datetime', 'legend', 'won']) # Convert list of lists to dataframe
  usesdf['datetime'] = pd.to_datetime(usesdf['datetime']) # Convert datetime column to actual datetime
  usesdf.sort_values(by='datetime', inplace=True) # Sort by datetime
  usesdf.reset_index(drop=True, inplace=True) # Reset indices

  daily_counts = usesdf.groupby('datetime').size().reset_index(name='count')
  daily_counts = daily_counts.set_index('datetime')
  full_date_range = pd.date_range(start=daily_counts.index.min(), end=daily_counts.index.max(), freq='D')
  daily_counts = daily_counts.reindex(full_date_range).fillna(0)
  # daily_counts

  plt.figure(figsize=(15,5))
  # monthly_counts = all_counts[all_counts['project']==project]
  plt.plot(daily_counts.index, daily_counts["count"], marker="o", linestyle="-")
  plt.xlabel("Date")
  plt.ylabel("Uses")
  plt.title(legend)
  plt.grid(True)
  plt.tight_layout()
  ax = plt.gca()
  for label in ax.xaxis.get_ticklabels()[::2]:
      label.set_visible(False)
  plt.xticks(rotation=45, fontsize=7)

In [ ]:

years = ['ALL'] + list(data['year'].unique())

@interact(
    legend=legends,
    fill=[True,False],
    year=years
)
def plot_legend(legend=legends[0], fill=True, year=years[0]):
    matches = data

    if year != 'ALL':
      matches = matches[matches['year'] == year]

    uses = []

    for row in matches.itertuples(index=True):
      # Record winner of the match as an integer
      if "1" in row.winner:
        winner = 1
      elif "2" in row.winner:
        winner = 2

      p1legends = list(set(list(filter(bool, [string.strip() for string in row.player_1_legends.split(",")])))) # Grab P1's legends and remove whitespace/empties/duplicates
      if legend in p1legends:
        uses.append([row.match_id, row.date, legend, winner == 1])

      p2legends = list(set(list(filter(bool, [string.strip() for string in row.player_2_legends.split(",")])))) # Grab P2's legends and remove whitespace/empties/duplicates
      if legend in p2legends:
        uses.append([row.match_id, row.date, legend, winner == 2])

    usesdf = pd.DataFrame(uses, columns=['match_id', 'datetime', 'legend', 'won']) # Convert list of lists to dataframe
    usesdf['datetime'] = pd.to_datetime(usesdf['datetime']) # Convert datetime column to actual datetime
    usesdf.sort_values(by='datetime', inplace=True) # Sort by datetime
    usesdf.reset_index(drop=True, inplace=True) # Reset indices

    daily_counts = usesdf.groupby('datetime').size().reset_index(name='count')
    daily_counts = daily_counts.set_index('datetime')

    if fill:
      full_date_range = pd.date_range(start=daily_counts.index.min(), end=daily_counts.index.max(), freq='D')
      daily_counts = daily_counts.reindex(full_date_range).fillna(0)

    plt.figure(figsize=(15,5))
    # monthly_counts = all_counts[all_counts['project']==project]
    plt.plot(daily_counts.index, daily_counts["count"], marker="o", linestyle="-")
    plt.xlabel("Date")
    plt.ylabel("Uses")
    plt.title(legend)
    plt.grid(True)
    plt.tight_layout()
    ax = plt.gca()
    for label in ax.xaxis.get_ticklabels()[::2]:
        label.set_visible(False)
    plt.xticks(rotation=45, fontsize=7)
    plt.show()